In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [48]:
# Load all data (except unknown/manual testing)
# Parsing + Basic 5x+ each, and reruns (such that all tests have 5+ results)
# Ubuntu 22.04, Android 11, MacOS 14.3.1, iPadOS 17.3.1
# All runs performed between 2024-03-04 and 2024-03-NN
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [49]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (1000s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

In [50]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [51]:
# Remove other browsers/os
df = df.loc[~df["browser_id"].isin([30, 27, 26, 54, 28])]

In [52]:
# Ignore MacOS for now as is is not finished yet?!
# df_m = df
# df = df.loc[~df["browser_id"].isin([51])]

In [53]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"], observed=True)["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,intent,65,brave,1.62.165,Android 11,real,1100804
1,intent,64,firefox_beta,123.0b9,Android 11,real,1015902
2,intent,62,chrome,121.0.6167.180,Android 11,real,922650
3,intent,63,opera,80.5.4244.78163,Android 11,real,907650
4,intent,60,chrome,122.0.6261.89,iPadOS 17.3.1,real,902096
5,selenium,47,chrome,122,Ubuntu 22.04,headless-new,896618
6,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,896230
7,selenium,61,firefox,123,Ubuntu 22.04,headless,896166
8,selenium,49,firefox,121,Ubuntu 22.04,headless,896166
9,selenium,44,edge,121,Ubuntu 22.04,headless-new,896152


# Error + Timeout Analysis
- Iterative design:
  - We use higher timeouts for certain tests that timed out more often in debug runs
  - We use higher timeouts for Android
  - We use higher timeouts for the repeat runs
- Small number of random timeouts in both parsing/basic mode
- Very small number of not-run in Opera and Brave Android and MacOS Safari (all HSTS)

In [54]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status         
brave Android 11 1.62.165 intent real              0            1100799
                                                   3                  5
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0             896222
                                                   2                  8
chrome Android 11 121.0.6167.180 intent real       0             922649
                                                   2                  1
chrome Ubuntu 22.04 122 selenium headless-new      0             896601
                                                   2                 17
chrome iPadOS 17.3.1 122.0.6261.89 intent real     0             902095
                                                   2                  1
edge Ubuntu 22.04 121 selenium headless-new        0             896151
                                                   2                  1
firefox Ubuntu 22.04 121 selenium headless         0             896165
                                                   2                  1
firefox Ubuntu 22.04 123 selenium headless         0             896165
                                                   2                  1
firefox_beta Android 11 123.0b9 intent real        0            1015711
                                                   2                191
opera Android 11 80.5.4244.78163 intent real       0             907611
                                                   2                 31
                                                   3                  8
safari macOS 14.3.1 17.3.1 selenium real           0             581363
                                                   2                 31
                                                   3                 28

In [55]:
# Teststatus == 2 (timeout)

# Rare timeouts in some browsers
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

with pd.option_context("display.max_rows", 80):
    display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
firefox_beta Android 11 123.0b9 intent real,191
opera Android 11 80.5.4244.78163 intent real,31
safari macOS 14.3.1 17.3.1 selenium real,31
chrome Ubuntu 22.04 122 selenium headless-new,17
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,8
chrome Android 11 121.0.6167.180 intent real,1
chrome iPadOS 17.3.1 122.0.6261.89 intent real,1
firefox Ubuntu 22.04 121 selenium headless,1
edge Ubuntu 22.04 121 selenium headless-new,1


count
resp_type status_code test_name               browser                                                  
basic     200         oac_iframe              firefox_beta Android 11 123.0b9 intent real            96
                      referrer_iframe         firefox_beta Android 11 123.0b9 intent real            42
          302         oac_iframe              firefox_beta Android 11 123.0b9 intent real            28
parsing   200         framing_iframe          safari macOS 14.3.1 17.3.1 selenium real               20
basic     200         fullscreen_iframe       opera Android 11 80.5.4244.78163 intent real           15
parsing   200         accesswindow_direct     firefox_beta Android 11 123.0b9 intent real            14
                      fullscreen_iframe       opera Android 11 80.5.4244.78163 intent real            9
basic     200         referrer_iframe         brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      6
parsing   200         upgradeHSTS_subdomain   safari macOS 14.3.1 17.3.1 selenium real                6
                      framing_iframe          firefox_beta Android 11 123.0b9 intent real             5
basic     200         fullscreen_iframe       chrome Ubuntu 22.04 122 selenium headless-new           5
                      framing_embed           chrome Ubuntu 22.04 122 selenium headless-new           4
                      framing_object          chrome Ubuntu 22.04 122 selenium headless-new           4
parsing   200         upgradeHSTS_direct      safari macOS 14.3.1 17.3.1 selenium real                4
basic     302         referrer_iframe         firefox_beta Android 11 123.0b9 intent real             4
parsing   200         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            4
basic     200         framing_iframe          chrome Ubuntu 22.04 122 selenium headless-new           3
                      fullscreen_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      2
          302         oac_window.open         firefox_beta Android 11 123.0b9 intent real             2
          200         referrer_iframe         chrome Ubuntu 22.04 122 selenium headless-new           1
                                              firefox Ubuntu 22.04 121 selenium headless              1
          302         framing_embed           chrome Android 11 121.0.6167.180 intent real            1
          200         referrer_iframe         firefox Ubuntu 22.04 123 selenium headless              1
parsing   200         accesswindow_direct     safari macOS 14.3.1 17.3.1 selenium real                1
                      fullscreen_iframe       edge Ubuntu 22.04 121 selenium headless-new             1
                      upgradeHSTS_subdomain   chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
          201         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1
          203         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1
          204         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1

In [56]:
# Teststatus == 3 (not-run)

display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

display(df.loc[(df["test_status"] == 3)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
safari macOS 14.3.1 17.3.1 selenium real,28
opera Android 11 80.5.4244.78163 intent real,8
brave Android 11 1.62.165 intent real,5


count
resp_type status_code test_name             browser                                            
parsing   200         upgradeHSTS_subdomain safari macOS 14.3.1 17.3.1 selenium real         21
                      upgradeHSTS_direct    safari macOS 14.3.1 17.3.1 selenium real          7
                      upgradeHSTS_subdomain brave Android 11 1.62.165 intent real             5
                      upgradeHSTS_direct    opera Android 11 80.5.4244.78163 intent real      4
                      upgradeHSTS_subdomain opera Android 11 80.5.4244.78163 intent real      4

# General Test Statistics

In [57]:
# Number of response_ids tested (should be 44415)
df.groupby("browser")["response_id"].nunique()

browser
brave Android 11 1.62.165 intent real                                  44415
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Android 11 121.0.6167.180 intent real                           44415
chrome Ubuntu 22.04 120 selenium headless-new                          44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44415
chrome iPadOS 17.3.1 122.0.6261.89 intent real                         44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44415
firefox Ubuntu 22.04 122 selenium headless                             44415
firefox Ubuntu 22.04 123 selenium headless                             44415
firefox_beta Android 11 123.0b9 intent real                            44415
opera Android 11 80.5.4244.78163 intent real                        

In [58]:
# Number or URLs is different due to repeats, run_ids, different numbers of popups
df.groupby("browser")["full_url"].nunique()

browser
brave Android 11 1.62.165 intent real                                  215098
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     26115
chrome Android 11 121.0.6167.180 intent real                           142105
chrome Ubuntu 22.04 120 selenium headless-new                           25174
chrome Ubuntu 22.04 121 selenium headless-new                           25183
chrome Ubuntu 22.04 122 selenium headless-new                           28966
chrome iPadOS 17.3.1 122.0.6261.89 intent real                          33244
edge Ubuntu 22.04 121 selenium headless-new                             25343
firefox Ubuntu 22.04 121 selenium headless                              25116
firefox Ubuntu 22.04 122 selenium headless                              25600
firefox Ubuntu 22.04 123 selenium headless                              25102
firefox_beta Android 11 123.0b9 intent real                            240928
opera Android 11 80.5.4244.78163 intent real            

In [59]:
# Number of test_ids
# Should be 10456 and 168774
df.loc[df["test_status"] == 0].groupby(["resp_type", "browser"])["test_id"].nunique()

resp_type  browser                                                            
basic      brave Android 11 1.62.165 intent real                                   10456
           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     10456
           chrome Android 11 121.0.6167.180 intent real                            10456
           chrome Ubuntu 22.04 120 selenium headless-new                           10456
           chrome Ubuntu 22.04 121 selenium headless-new                           10456
           chrome Ubuntu 22.04 122 selenium headless-new                           10456
           chrome iPadOS 17.3.1 122.0.6261.89 intent real                          10456
           edge Ubuntu 22.04 121 selenium headless-new                             10456
           firefox Ubuntu 22.04 121 selenium headless                              10456
           firefox Ubuntu 22.04 122 selenium headless                              10456
           firefox Ubuntu 22.04

In [60]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- How many tests have more than one result for the 5+ repetitions in a browser?
- Basic:
    - total of 285/(10456*14) have different results
    - subresourceloadingCOEP_img is the worst offender (especially random behavior Firefox bug)
    - should be mostly timeout related and "fixed" by majority voting
- Parsing:
    - total of 1443/(168774*12) have different results
    - subresourceloadingCOEP_img + hsts, ...
    - Safari (MacOS + iPad a bit more than others?)
    - ...

In [61]:
# Remove all tests that timed out (test_status == 0), each test should have at least 5 results that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [62]:
details = False
for group_name, resp_type_group in df.groupby("resp_type"):
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    with pd.option_context("display.max_rows", 100):
        display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()

    if not details:
        continue
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)

basic
285 tests have different outcomes!


,count
browser,
firefox_beta Android 11 123.0b9 intent real,95
opera Android 11 80.5.4244.78163 intent real,39
firefox Ubuntu 22.04 123 selenium headless,35
firefox Ubuntu 22.04 121 selenium headless,32
firefox Ubuntu 22.04 122 selenium headless,31
brave Android 11 1.62.165 intent real,19
chrome Android 11 121.0.6167.180 intent real,8
chrome Ubuntu 22.04 120 selenium headless-new,8
safari macOS 14.3.1 17.3.1 selenium real,7


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img firefox_beta Android 11 123.0b9 intent real            95
                           firefox Ubuntu 22.04 123 selenium headless             34
                           firefox Ubuntu 22.04 121 selenium headless             32
                           firefox Ubuntu 22.04 122 selenium headless             30
referrer_iframe            opera Android 11 80.5.4244.78163 intent real           17
accesswindow_direct        opera Android 11 80.5.4244.78163 intent real           12
oac_window.open            opera Android 11 80.5.4244.78163 intent real           10
                           brave Android 11 1.62.165 intent real                   9
perfAPI_img                chrome Ubuntu 22.04 120 selenium headless-new           7
subresourceloadingCOEP_img safari macOS 14.3.1 17.3.1 selenium real                7
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      5
subresourceloadingCOEP_img chrome iPadOS 17.3.1 122.0.6261.89 intent real          5
upgradeHSTS_direct         brave Android 11 1.62.165 intent real                   4
referrer_iframe            brave Android 11 1.62.165 intent real                   4
                           chrome Android 11 121.0.6167.180 intent real            3
accesswindow_direct        chrome Android 11 121.0.6167.180 intent real            3
oac_window.open            chrome Android 11 121.0.6167.180 intent real            2
upgradeHSTS_subdomain      brave Android 11 1.62.165 intent real                   2
framing_embed              firefox Ubuntu 22.04 123 selenium headless              1
framing_object             firefox Ubuntu 22.04 122 selenium headless              1
referrer_iframe            chrome Ubuntu 22.04 121 selenium headless-new           1
                           chrome Ubuntu 22.04 120 selenium headless-new           1

parsing
1443 tests have different outcomes!


,count
browser,
safari macOS 14.3.1 17.3.1 selenium real,316
chrome iPadOS 17.3.1 122.0.6261.89 intent real,161
firefox Ubuntu 22.04 123 selenium headless,121
brave Android 11 1.62.165 intent real,119
chrome Ubuntu 22.04 122 selenium headless-new,113
chrome Ubuntu 22.04 121 selenium headless-new,112
chrome Android 11 121.0.6167.180 intent real,97
chrome Ubuntu 22.04 120 selenium headless-new,83
opera Android 11 80.5.4244.78163 intent real,78


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img chrome iPadOS 17.3.1 122.0.6261.89 intent real        144
                           safari macOS 14.3.1 17.3.1 selenium real               89
fetch_GET                  safari macOS 14.3.1 17.3.1 selenium real               73
subresourceloadingCOEP_img chrome Ubuntu 22.04 121 selenium headless-new          65
                           firefox Ubuntu 22.04 121 selenium headless             56
upgradeHSTS_direct         chrome Ubuntu 22.04 120 selenium headless-new          55
                           brave Android 11 1.62.165 intent real                  54
subresourceloadingCOEP_img firefox Ubuntu 22.04 123 selenium headless             53
upgradeHSTS_direct         firefox Ubuntu 22.04 123 selenium headless             49
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             48
                           firefox_beta Android 11 123.0b9 intent real            45
                           chrome Ubuntu 22.04 122 selenium headless-new          44
upgradeHSTS_direct         chrome Android 11 121.0.6167.180 intent real           44
imgloading_iframe          chrome Ubuntu 22.04 121 selenium headless-new          42
upgradeHSTS_subdomain      safari macOS 14.3.1 17.3.1 selenium real               37
script_execution_iframe    safari macOS 14.3.1 17.3.1 selenium real               35
upgradeHSTS_direct         safari macOS 14.3.1 17.3.1 selenium real               27
subresourceloadingCOEP_img edge Ubuntu 22.04 121 selenium headless-new            24
fetch_TEST                 safari macOS 14.3.1 17.3.1 selenium real               24
upgradeHSTS_direct         opera Android 11 80.5.4244.78163 intent real           21
upgradeHSTS_subdomain      firefox Ubuntu 22.04 123 selenium headless             19
framing_iframe             chrome Ubuntu 22.04 122 selenium headless-new          19
subresourceloadingCOEP_img chrome Android 11 121.0.6167.180 intent real           19
framing_iframe             opera Android 11 80.5.4244.78163 intent real           19
upgradeHSTS_subdomain      brave Android 11 1.62.165 intent real                  18
subresourceloadingCOEP_img chrome Ubuntu 22.04 120 selenium headless-new          17
upgradeHSTS_subdomain      chrome Ubuntu 22.04 122 selenium headless-new          15
subresourceloadingCOEP_img brave Android 11 1.62.165 intent real                  14
script_execution_iframe    firefox Ubuntu 22.04 122 selenium headless             14
                           brave Android 11 1.62.165 intent real                  13
upgradeHSTS_subdomain      chrome Android 11 121.0.6167.180 intent real           13
                           opera Android 11 80.5.4244.78163 intent real           12
upgradeHSTS_direct         chrome Ubuntu 22.04 122 selenium headless-new          12
imgloading_iframe          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     11
                           firefox Ubuntu 22.04 122 selenium headless             10
fetch_GET                  brave Android 11 1.62.165 intent real                  10
                           opera Android 11 80.5.4244.78163 intent real           10
perfAPI_img                chrome Ubuntu 22.04 122 selenium headless-new          10
imgloading_iframe          safari macOS 14.3.1 17.3.1 selenium real                9
script_execution_iframe    opera Android 11 80.5.4244.78163 intent real            9
framing_iframe             safari macOS 14.3.1 17.3.1 selenium real                9
imgloading_iframe          chrome Ubuntu 22.04 120 selenium headless-new           9
script_execution_iframe    chrome Android 11 121.0.6167.180 intent real            8
accesswindow_direct        brave Android 11 1.62.165 intent real                   8
fullscreen_iframe          safari macOS 14.3.1 17.3.1 selenium real                8
upgradeHSTS_direct         chrome iPadOS 17.3.1 122.0.6261.89 intent real          7

## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [63]:
# How many outcomes did we collect for each test?!
# Should be 5+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    # d = d.loc[d["org_scheme"] == "http"]
    res = d.groupby(["test_id"], observed=True)["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()    
    counts = counts.rename(columns={"count": group})
    display(counts)
    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

,brave Android 11 1.62.165 intent real
count,
5,163424
24,1657
6,1598
9,1191
8,1014
7,965
10,963
14,927
34,718


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
count,
5,179158
6,72


,chrome Android 11 121.0.6167.180 intent real
count,
5,173460
14,1477
6,1142
10,874
7,854
8,623
9,583
13,169
12,22


,chrome Ubuntu 22.04 120 selenium headless-new
count,
5,179230


,chrome Ubuntu 22.04 121 selenium headless-new
count,
5,179230


...

,firefox Ubuntu 22.04 123 selenium headless
count,
5,179215
6,15


,firefox_beta Android 11 123.0b9 intent real
count,
5,174067
20,836
21,812
14,510
50,455
19,435
22,353
13,281
51,260


,opera Android 11 80.5.4244.78163 intent real
count,
5,176904
10,1692
9,391
13,113
6,55
14,30
12,15
8,15
7,10


,safari macOS 14.3.1 17.3.1 selenium real
count,
2,81595
5,59612
3,16860
4,12080
1,6762
6,2028
8,232
7,61


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
count,,,,,,,,,,,,,,
5,163424.0,179158.0,173460.0,179230.0,179230.0,178779.0,175047.0,179229.0,179215.0,179230.0,179215.0,174067.0,176904.0,59612.0
24,1657.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
6,1598.0,72.0,1142.0,NaN,NaN,451.0,3485.0,1.0,15.0,NaN,15.0,7.0,55.0,2028.0
9,1191.0,NaN,583.0,NaN,NaN,NaN,122.0,NaN,NaN,NaN,NaN,12.0,391.0,NaN
8,1014.0,NaN,623.0,NaN,NaN,NaN,221.0,NaN,NaN,NaN,NaN,5.0,15.0,232.0
7,965.0,NaN,854.0,NaN,NaN,NaN,245.0,NaN,NaN,NaN,NaN,6.0,10.0,61.0
10,963.0,NaN,874.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,13.0,1692.0,NaN
14,927.0,NaN,1477.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,510.0,30.0,NaN
34,718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [64]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "min", "max", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))
    display(res.loc[res["mean"] > 0].tail(20))


,mean,min,max,count,sum
count,448.000000,448.000000,448.000000,448.000000,448.000000
mean,5.094343,4.888393,7.875000,5600.937500,28124.959821
std,0.864184,0.638825,8.615708,10292.443524,51985.281801
min,2.124908,1.000000,3.000000,28.000000,140.000000
25%,5.000000,5.000000,5.000000,284.000000,1430.000000
50%,5.000000,5.000000,5.000000,1680.000000,8400.000000
75%,5.010216,5.000000,6.000000,5487.500000,27970.000000
max,16.491760,5.000000,54.000000,48354.000000,342582.000000


mean  \
browser                                        resp_type test_name                            
firefox_beta Android 11 123.0b9 intent real    parsing   oac_window.open          16.491760   
                                                         accesswindow_direct      12.941453   
brave Android 11 1.62.165 intent real          parsing   script_execution_iframe  10.913793   
firefox_beta Android 11 123.0b9 intent real    parsing   imgloading_iframe         9.383395   
brave Android 11 1.62.165 intent real          parsing   oac_window.open           7.273690   
                                                         fetch_TEST                7.084874   
                                                         fetch_GET                 7.084874   
chrome Android 11 121.0.6167.180 intent real   parsing   oac_window.open           6.691583   
opera Android 11 80.5.4244.78163 intent real   basic     accesswindow_direct       6.522727   
safari macOS 14.3.1 17.3.1 selenium real       basic     fetch_TEST                6.500000   
                                                         fetch_GET                 6.500000   
firefox_beta Android 11 123.0b9 intent real    basic     oac_iframe                6.473214   
chrome Android 11 121.0.6167.180 intent real   parsing   accesswindow_direct       6.429593   
brave Android 11 1.62.165 intent real          basic     upgradeHSTS_direct        6.357143   
                                                         upgradeHSTS_subdomain     6.357143   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     accesswindow_direct       6.045455   
brave Android 11 1.62.165 intent real          parsing   accesswindow_direct       5.725527   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     oac_iframe                5.678571   
                                                         oac_window.open           5.678571   
firefox_beta Android 11 123.0b9 intent real    parsing   fullscreen_iframe         5.663402   

                                                                                  min  \
browser                                        resp_type test_name                      
firefox_beta Android 11 123.0b9 intent real    parsing   oac_window.open            5   
                                                         accesswindow_direct        5   
brave Android 11 1.62.165 intent real          parsing   script_execution_iframe    5   
firefox_beta Android 11 123.0b9 intent real    parsing   imgloading_iframe          5   
brave Android 11 1.62.165 intent real          parsing   oac_window.open            5   
                                                         fetch_TEST                 5   
                                                         fetch_GET                  5   
chrome Android 11 121.0.6167.180 intent real   parsing   oac_window.open            5   
opera Android 11 80.5.4244.78163 intent real   basic     accesswindow_direct        5   
safari macOS 14.3.1 17.3.1 selenium real       basic     fetch_TEST                 5   
                                                         fetch_GET                  5   
firefox_beta Android 11 123.0b9 intent real    basic     oac_iframe                 5   
chrome Android 11 121.0.6167.180 intent real   parsing   accesswindow_direct        5   
brave Android 11 1.62.165 intent real          basic     upgradeHSTS_direct         5   
                                                         upgradeHSTS_subdomain      5   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     accesswindow_direct        5   
brave Android 11 1.62.165 intent real          parsing   accesswindow_direct        5   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     oac_iframe                 5   
                                                         oac_window.open            5   
firefox_beta Android 11 123.0b9 intent real    parsing   fullscreen_iframe          5   

                                     

mean  \
browser                                        resp_type test_name                              
edge Ubuntu 22.04 121 selenium headless-new    parsing   upgradeHSTS_subdomain       5.000000   
                                                         upgradeHSTS_direct          5.000000   
                                                         subresourceloadingCORP_img  5.000000   
                                                         subresourceloadingCOEP_img  5.000000   
                                                         script_execution_iframe     5.000000   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     framing_iframe              5.000000   
safari macOS 14.3.1 17.3.1 selenium real       parsing   subresourceloadingCORP_img  4.982127   
                                                         upgradeHSTS_direct          4.121992   
                                                         upgradeHSTS_subdomain       4.119585   
                                                         referrer_iframe             3.771541   
                                                         perfAPI_img                 3.400999   
                                                         fetch_GET                   3.169996   
                                                         fetch_TEST                  3.169996   
                                                         oac_window.open             3.031195   
                                                         imgloading_iframe           3.000000   
                                                         subresourceloadingCOEP_img  2.986159   
                                                         fullscreen_iframe           2.820747   
                                                         framing_iframe              2.574601   
                                                         accesswindow_direct         2.551770   
                                                         script_execution_iframe     2.124908   

                                                                                     min  \
browser                                        resp_type test_name                         
edge Ubuntu 22.04 121 selenium headless-new    parsing   upgradeHSTS_subdomain         5   
                                                         upgradeHSTS_direct            5   
                                                         subresourceloadingCORP_img    5   
                                                         subresourceloadingCOEP_img    5   
                                                         script_execution_iframe       5   
chrome iPadOS 17.3.1 122.0.6261.89 intent real basic     framing_iframe                5   
safari macOS 14.3.1 17.3.1 selenium real       parsing   subresourceloadingCORP_img    4   
                                                         upgradeHSTS_direct            1   
                                                         upgradeHSTS_subdomain         1   
                                                         referrer_iframe               2   
                                                         perfAPI_img                   1   
                                                         fetch_GET                     1   
                                                         fetch_TEST                    1   
                                                         oac_window.open               2   
                                                         imgloading_iframe             1   
                                                         subresourceloadingCOEP_img    1   
                                                         fullscreen_iframe             2   
                                                         framing_iframe                1   
                                                         accesswindow_direct           1   
                      

In [65]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 100):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))

,mean,count,sum
count,448.000000,448.000000,448.000000
mean,0.002886,5600.937500,3.857143
std,0.017268,10292.443524,13.241084
min,0.000000,28.000000,0.000000
25%,0.000000,284.000000,0.000000
50%,0.000000,1680.000000,0.000000
75%,0.000000,5487.500000,0.000000
max,0.269886,48354.000000,144.000000


mean  \
browser                                            resp_type test_name                              
firefox_beta Android 11 123.0b9 intent real        basic     subresourceloadingCOEP_img  0.269886   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_direct          0.142857   
firefox Ubuntu 22.04 123 selenium headless         basic     subresourceloadingCOEP_img  0.096591   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img  0.090909   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img  0.085227   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_subdomain       0.071429   
opera Android 11 80.5.4244.78163 intent real       basic     accesswindow_direct         0.068182   
                                                             oac_window.open             0.044643   
brave Android 11 1.62.165 intent real              basic     oac_window.open             0.040179   
chrome Ubuntu 22.04 120 selenium headless-new      basic     perfAPI_img                 0.033654   
opera Android 11 80.5.4244.78163 intent real       basic     referrer_iframe             0.033203   
chrome iPadOS 17.3.1 122.0.6261.89 intent real     parsing   subresourceloadingCOEP_img  0.027304   
safari macOS 14.3.1 17.3.1 selenium real           basic     subresourceloadingCOEP_img  0.019886   
chrome Android 11 121.0.6167.180 intent real       basic     accesswindow_direct         0.017045   
safari macOS 14.3.1 17.3.1 selenium real           parsing   subresourceloadingCOEP_img  0.016875   
chrome iPadOS 17.3.1 122.0.6261.89 intent real     basic     subresourceloadingCOEP_img  0.014205   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img  0.012325   
firefox Ubuntu 22.04 121 selenium headless         parsing   subresourceloadingCOEP_img  0.010618   
firefox Ubuntu 22.04 123 selenium headless         parsing   subresourceloadingCOEP_img  0.010049   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     referrer_iframe             0.009766   

                                                                                         count  \
browser                                            resp_type test_name                           
firefox_beta Android 11 123.0b9 intent real        basic     subresourceloadingCOEP_img    352   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_direct             28   
firefox Ubuntu 22.04 123 selenium headless         basic     subresourceloadingCOEP_img    352   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img    352   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img    352   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_subdomain          28   
opera Android 11 80.5.4244.78163 intent real       basic     accesswindow_direct           176   
                                                             oac_window.open               224   
brave Android 11 1.62.165 intent real              basic     oac_window.open               224   
chrome Ubuntu 22.04 120 selenium headless-new      basic     perfAPI_img                   208   
opera Android 11 80.5.4244.78163 intent real       basic     referrer_iframe               512   
chrome iPadOS 17.3.1 122.0.6261.89 intent real     parsing   subresourceloadingCOEP_img   5274   
safari macOS 14.3.1 17.3.1 selenium real           basic     subresourceloadingCOEP_img    352   
chrome Android 11 121.0.6167.180 intent real       basic     accesswindow_direct           176   
safari macOS 14.3.1 17.3.1 selenium real           parsing   subresourceloadingCOEP_img   5274   
chrome iPadOS 17.3.1 122.0.6261.89 intent real     basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
firefox 

# Browser differences!
- First perform majority voting
- Then diff the browsers

In [80]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [81]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)

df = df.groupby(["test_id", "browser"], observed=True)["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 12599982
Only one row for each test (per browser; majority voting) 2509220


In [82]:
# Merge back additional required properties!
# ["name", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "resp_type", "label", 'org_origin', 'resp_origin', 'status_code', 'raw_header']

df = df.merge(df_stab.drop_duplicates(subset=["test_id", "browser"]), on=["test_id", "browser"], how="left", suffixes=["", "_ignore"])


In [87]:
# Tests with same mode?! maybe rerun again?!
with pd.option_context("display.max_colwidth", None):
    # Firefox subresourceloadingCOEP_img
    # Ignore, indeterminisitc behavior.
    display(df.loc[(df["browser"] == 'firefox_beta Android 11 123.0b9 intent real') & (df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray)))][["outcome_str", "full_url"]])

    # Brave (Android) oac_window.open
    # Rerun one URL!
    display(df.loc[(df["browser"] == 'brave Android 11 1.62.165 intent real') & (df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray)))][["outcome_str", "full_url"]])

    # Opera (Android) ugrade_HSTS_direct
    # Rerun one URL!
    display(df.loc[(df["browser"] == 'opera Android 11 80.5.4244.78163 intent real') & (df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray)))][["outcome_str", "full_url"]])

    # Safari (MacOS) various
    # Still ongoing, hopefully fixed after 5+ repetitions are finished
    display(df.loc[(df["browser"] == 'safari macOS 14.3.1 17.3.1 selenium real') & (df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray)))][["outcome_str", "full_url"]])
    

,outcome_str,full_url
2177221,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=58&last_id=58&scheme=http&run_id=645058ab2c9143599e445a365f502857
2177235,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=59&last_id=59&scheme=http&run_id=141f1711bc294e1db4f626281598088a
2177319,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=65&last_id=65&scheme=http&run_id=50b6106d332c437f8a76f52d34f6bbb6
2177641,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=66&last_id=66&scheme=http&run_id=db5f71465546446eacf64876591928e2
2177767,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=64&last_id=64&scheme=http&run_id=f0376075d30b402585774303b11dcda4
2177795,"[{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}]",http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=66&last_id=66&scheme=http&run_id=db5f71465546446eacf64876591928e2
2215385,"[{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}, {'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}]",https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=10&resp_type=basic&browser_id=64&label=COEP&first_id=59&last_id=59&scheme=https&run_id=c38e1479189245d8a7b5ec707d79f6a0
2215847,"[{'image-events': {'swag.jpg': 'error', 'swag-same-site.

,outcome_str,full_url
1897294,"[window.originAgentCluster: false, window.originAgentCluster: true]",https://sub.headers.websec.saarland/_hp/tests/originAgentCluster-oac.sub.html?timeout=20&resp_type=basic&browser_id=65&label=OAC&first_id=156&last_id=156&scheme=https&first_popup=7&last_popup=7&run_no_popup=no&run_id=aecff641597e482792e215fd46c23478


,outcome_str,full_url
2404148,"[Fetch timed out, {'response.redirected': False}]",http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?timeout=20&resp_type=parsing&browser_id=63&label=HSTS&first_id=32198&last_id=32198&scheme=http&run_id=da8b148fd5254f4d9d3904003427392f


,outcome_str,full_url
146719,"[{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}, {'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}]",http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?timeout=5&resp_type=parsing&browser_id=51&label=CORS-ACEH&first_id=14966&last_id=14970&scheme=http
146733,"[{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}, {'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}]",http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?timeout=5&resp_type=parsing&browser_id=51&label=CORS-ACEH&first_id=14966&last_id=14970&scheme=http
146747,"[{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}, {'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}]",http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?timeout=5&resp_type=parsing&browser_id=51&label=CORS-ACEH&first_id=14966&last_id=14970&scheme=http
146761,"[{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}, {'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}]",http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?timeout=5&resp_type=parsing&browser_id=51&label=CORS-ACEH&first_id=14966&last_id=14970&scheme=http
149617,"[{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}, {'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}]",http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?timeout=5&resp_type=parsing&browser_id=51&label=CORS-ACEH&first_id=15171&last_id=15175&scheme=http
...,...,...
2385795,"[Fetch timed out, {'response.redirected': False}]",http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?timeout=10&resp_type=parsing&browser_id=51&label=HSTS&first_id=30887&last_id=30887&scheme=http
2385963,"[Fetch timed out, {'response.redirected': False}]",http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?timeout=10&resp_type=parsing&browser_id=51&label=HSTS&first_id=30899&last_id=30899&scheme=http
2386691,"[Fetch timed out, {'response.redirected': False}]",http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?timeout=10&resp_type=parsing&browser_id=51&label=HSTS&first_id=30951&last_id=30951&scheme=http
2465749,"[Fetch timed out, {'response.redirected': False}]",http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?timeout=10&resp_type=parsing&browser_id=51&label=HSTS&first_id=29922&last_id=29922&scheme=http


In [258]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# If we have exactly 5 repetitions for each test, the issue cannot exist!
# However, our repeat system only enables us to have 5+ results for each test
print("Tests with the same mode (highest frequency outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees and other summaries!)
# These are unstable tests that might indicate random behavior in the browser (e.g., firefox COEP_img)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequency outcome):


browser                                       resp_type  test_name                 
brave Android 11 1.62.165 intent real         basic      oac_window.open                1
firefox_beta Android 11 123.0b9 intent real   basic      subresourceloadingCOEP_img     8
opera Android 11 80.5.4244.78163 intent real  parsing    upgradeHSTS_direct             1
safari macOS 14.3.1 17.3.1 selenium real      parsing    fetch_GET                     19
                                                         fetch_TEST                     7
                                                         framing_iframe                 7
                                                         fullscreen_iframe              7
                                                         oac_window.open                4
                                                         script_execution_iframe       30
                                                         subresourceloadingCOEP_img    59
                

In [71]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "test_id"], observed=True)
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,170900
2,8238
3,92


In [72]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
basic     subresourceloadingCORP_object direct            176   176  1.000000
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    direct            224   144  0.642857
                                        sandbox           224   144  0.642857
          subresourceloadingCOEP_img    direct            176    88  0.500000
          referrer_iframe               window.open       256   120  0.468750
          fullscreen_iframe             direct            384   142  0.369792
          framing_embed                 direct            560   186  0.332143
          framing_object                direct            560   186  0.332143
          fullscreen_iframe             child_allow       384   113  0.294271
          perfAPI_img                   direct            208    55  0.264423
          framing_object                sandbox           560   120  0.214286
                                        nested            560   120  0.214286
          framing_embed                 nested            560   120  0.214286
                                        sandbox           560   120  0.214286
          accesswindow_direct           direct            176    28  0.159091
          script_execution_iframe       sandbox           272    21  0.077206
          referrer_iframe               iframe            256    18  0.070312
          fullscreen_iframe             child             384    23  0.059896
          imgloading_iframe             direct            272    13  0.047794
parsing   upgradeHSTS_direct            direct           6648   317  0.047684
          fullscreen_iframe             child_allow      7760   355  0.045747
          imgloading_iframe             direct           5420   230  0.042435
          script_execution_iframe       direct           5452   216  0.039618
basic     upgradeHSTS_direct            direct             28     1  0.035714
parsing   fullscreen_iframe             direct           7760   272  0.035052
          subresourceloadingCOEP_img    direct           5274   151  0.028631
basic     subresourceloadingCOEP_img    sandbox           176     4  0.022727
parsing   framing_iframe                direct          17560   365  0.020786
          referrer_iframe               iframe           5594   101  0.018055
          upgradeHSTS_subdomain         subdomain        6648   105  0.015794
basic     framing_iframe                sandbox           560     8  0.014286
parsing   perfAPI_img                   direct           3202    33  0.010306
          subresourceloadingCORP_img    direct           6714    68  0.010128
          accesswindow_direct           direct           5312    44  0.008283
          framing_iframe                nested          17560   144  0.008200
          fetch_GET                     simple          16118   110  0.006825
                                        custom_headers  16118   107  0.006639
          fetch_TEST                    custom_method   16118   107  0.006639
          fetch_GET                     credentials     16118    92  0.005708
basic     subresourceloadingCORP_img    direct            176     1  0.005682
          fetch_GET                     credentials        96     0  0.000000
                                        simple             96     0  0.000000
                                        custom_headers     96     0  0.000000
          fullscreen_iframe             child_sandbox     384     0  0.000000
          framing_iframe                nested            560     0  0.000000
                                        direct            560     0  0.000000
          fetch_TEST                    custom_method      96     0  0.000000
          upgradeHSTS_subdomain         subdomain          28     0  0.000000


In [73]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])


all_tests_should_be =  df_org.loc[df_org["test_status"] == 0]["test_id"].nunique() * df_org["browser"].nunique() # (num parsing tests + num_basic_tests) * num_browsers (168774+10456)
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 2509220, should be: 2509220, Rows with more than one outcome: 116620, Tests with more than one outcome: 8330


In [74]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
10


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[chrome iPadOS 17.3.1 122.0.6261.89 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, opera Android 11 80.5.4244.78163 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[chrome iPadOS 17.3.1 122.0.6261.89 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, opera Android 11 80.5.4244.78163 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.62.165 intent real, chrome Android 11 121.0.6167.180 intent real, chrome iPadOS 17.3.1 122.0.6261.89 intent real, firefox_beta Android 11 123.0b9 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, chrome iPadOS 17.3.1 122.0.6261.89 intent real, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real, safari macOS 14.3.1 17.3.1 selenium real]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"


...

,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, opera Android 11 80.5.4244.78163 intent real]"
{'response.redirected': True},"[chrome iPadOS 17.3.1 122.0.6261.89 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
TypeError: Failed to fetch,"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, opera Android 11 80.5.4244.78163 intent real]"
{'response.redirected': False},"[chrome iPadOS 17.3.1 122.0.6261.89 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, chrome iPadOS 17.3.1 122.0.6261.89 intent real, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, safari macOS 14.3.1 17.3.1 selenium real]"
{'response.redirected': True},[opera Android 11 80.5.4244.78163 intent real]


,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
{'response.redirected': True},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, chrome iPadOS 17.3.1 122.0.6261.89 intent real, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"
{'response.redirected': True},"[chrome iPadOS 17.3.1 122.0.6261.89 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, safari macOS 14.3.1 17.3.1 selenium real]"


In [75]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

('accesswindow_direct', 'direct')


,['firefox_beta Android 11 123.0b9 intent real'],['safari macOS 14.3.1 17.3.1 selenium real'],"['chrome iPadOS 17.3.1 122.0.6261.89 intent real', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless']",['brave Android 11 1.62.165 intent real'],['chrome Android 11 121.0.6167.180 intent real'],['opera Android 11 80.5.4244.78163 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
{'window.open.opener': 'null'},12,16,24,41,45,49,56
"{'window.open.opener': 'object ""[object Window]""'}",60,56,48,31,27,23,16



('fetch_GET', 'credentials')


,"['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']","['chrome iPadOS 17.3.1 122.0.6261.89 intent real', 'safari macOS 14.3.1 17.3.1 selenium real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']",['safari macOS 14.3.1 17.3.1 selenium real'],['chrome iPadOS 17.3.1 122.0.6261.89 intent real']
"{'error': 'null', 'headers': 'content-length,'}",3.0,9.0,10,NaN,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,31.0,20,NaN,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",70.0,NaN,22,46.0,52.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,40,6.0,NaN



('fetch_GET', 'custom_headers')


...

,"['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['chrome iPadOS 17.3.1 122.0.6261.89 intent real', 'safari macOS 14.3.1 17.3.1 selenium real']"
error,44.0,76.0,NaN
load,132.0,100.0,NaN
timeout: no event fired,NaN,NaN,176.0



('upgradeHSTS_direct', 'direct')


,['opera Android 11 80.5.4244.78163 intent real'],"['brave Android 11 1.62.165 intent real', 'safari macOS 14.3.1 17.3.1 selenium real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']",['safari macOS 14.3.1 17.3.1 selenium real'],['chrome iPadOS 17.3.1 122.0.6261.89 intent real'],['brave Android 11 1.62.165 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['chrome Android 11 121.0.6167.180 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless']",['firefox_beta Android 11 123.0b9 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']","['chrome iPadOS 17.3.1 122.0.6261.89 intent real', 'safari macOS 14.3.1 17.3.1 selenium real']"
Fetch timed out,1.0,5.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{'response.redirected': False},77.0,NaN,NaN,NaN,45.0,50.0,76.0,78.0,79.0,290.0,292.0,NaN,NaN
TypeError: Failed to fetch,NaN,NaN,9.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{'response.redirected': True},NaN,NaN,NaN,NaN,NaN,NaN,210.0,NaN,212.0,7.0,5.0,213.0,268.0



('upgradeHSTS_subdomain', 'subdomain')


,['safari macOS 14.3.1 17.3.1 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']",['opera Android 11 80.5.4244.78163 intent real'],"['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['chrome iPadOS 17.3.1 122.0.6261.89 intent real'],"['chrome iPadOS 17.3.1 122.0.6261.89 intent real', 'safari macOS 14.3.1 17.3.1 selenium real']"
Fetch timed out,3.0,12,NaN,NaN,NaN,NaN,NaN
{'response.redirected': False},43.0,81,NaN,41.0,42.0,46.0,NaN
TypeError: Failed to fetch,NaN,9,27.0,NaN,NaN,NaN,NaN
{'response.redirected': True},NaN,3,NaN,37.0,36.0,NaN,59.0


In [76]:
# Simmilarity between browsers and versions!
# (Current issue: displays a difference if one browser has no result at all; in the end we have 5+ stable results for each tests, thus it should not be an issue anymore)

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            # TODO: use a custom Order (e.g., all Android, or some chromium versions next to each other)
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("resp_type"):
    display_overlap(group, show_all=True, name=name)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,152,82,101,102,102,6622,102,6963,6960,6958,6971,125,6730
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,152,0,161,141,142,142,6568,142,6896,6893,6891,6930,182,6684
chrome Android 11 121.0.6167.180 intent real,82,161,0,20,21,21,6603,21,6931,6928,6926,6939,50,6711
chrome Ubuntu 22.04 120 selenium headless-new,101,141,20,0,1,1,6609,1,6923,6920,6918,6957,46,6725
chrome Ubuntu 22.04 121 selenium headless-new,102,142,21,1,0,0,6610,0,6924,6921,6919,6958,47,6726
chrome Ubuntu 22.04 122 selenium headless-new,102,142,21,1,0,0,6610,0,6924,6921,6919,6958,47,6726
chrome iPadOS 17.3.1 122.0.6261.89 intent real,6622,6568,6603,6609,6610,6610,0,6610,2788,2785,2781,2812,6585,172
edge Ubuntu 22.04 121 selenium headless-new,102,142,21,1,0,0,6610,0,6924,6921,6919,6958,47,6726
firefox Ubuntu 22.04 121 selenium headless,6963,6896,6931,6923,6924,6924,2788,6924,0,27,33,90,6899,2822
firefox Ubuntu 22.04 122 selenium headless,6960,6893,6928,6920,6921,6921,2785,6921,27,0,38,83,6896,2823


basic


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,64,77,83,84,84,1966,84,1036,1036,1037,1060,107,2019
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,64,0,77,71,72,72,1986,72,1055,1055,1056,1079,106,2039
chrome Android 11 121.0.6167.180 intent real,77,77,0,6,7,7,1951,7,1007,1007,1008,1031,34,2004
chrome Ubuntu 22.04 120 selenium headless-new,83,71,6,0,1,1,1957,1,1013,1013,1014,1037,40,2010
chrome Ubuntu 22.04 121 selenium headless-new,84,72,7,1,0,0,1958,0,1014,1014,1015,1038,41,2011
chrome Ubuntu 22.04 122 selenium headless-new,84,72,7,1,0,0,1958,0,1014,1014,1015,1038,41,2011
chrome iPadOS 17.3.1 122.0.6261.89 intent real,1966,1986,1951,1957,1958,1958,0,1958,1166,1166,1167,1204,1927,53
edge Ubuntu 22.04 121 selenium headless-new,84,72,7,1,0,0,1958,0,1014,1014,1015,1038,41,2011
firefox Ubuntu 22.04 121 selenium headless,1036,1055,1007,1013,1014,1014,1166,1014,0,12,15,54,983,1117
firefox Ubuntu 22.04 122 selenium headless,1036,1055,1007,1013,1014,1014,1166,1014,12,0,15,54,983,1117


parsing


...

,brave Android 11 1.62.165 intent real,chrome iPadOS 17.3.1 122.0.6261.89 intent real,firefox Ubuntu 22.04 121 selenium headless
brave Android 11 1.62.165 intent real,0,176,32
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,176,32
chrome Android 11 121.0.6167.180 intent real,0,176,32
chrome Ubuntu 22.04 120 selenium headless-new,0,176,32
chrome Ubuntu 22.04 121 selenium headless-new,0,176,32
chrome Ubuntu 22.04 122 selenium headless-new,0,176,32
chrome iPadOS 17.3.1 122.0.6261.89 intent real,176,0,176
edge Ubuntu 22.04 121 selenium headless-new,0,176,32
firefox Ubuntu 22.04 121 selenium headless,32,176,0
firefox Ubuntu 22.04 122 selenium headless,32,176,0


upgradeHSTS_direct


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome iPadOS 17.3.1 122.0.6261.89 intent real,firefox Ubuntu 22.04 121 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,7,5,91,250,248,6,96
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,7,0,3,84,244,246,5,89
chrome Android 11 121.0.6167.180 intent real,5,3,0,87,247,245,4,92
chrome Ubuntu 22.04 120 selenium headless-new,7,0,3,84,244,246,5,89
chrome Ubuntu 22.04 121 selenium headless-new,7,0,3,84,244,246,5,89
chrome Ubuntu 22.04 122 selenium headless-new,7,0,3,84,244,246,5,89
chrome iPadOS 17.3.1 122.0.6261.89 intent real,91,84,87,0,282,284,89,5
edge Ubuntu 22.04 121 selenium headless-new,7,0,3,84,244,246,5,89
firefox Ubuntu 22.04 121 selenium headless,250,244,247,282,0,2,249,287
firefox Ubuntu 22.04 122 selenium headless,250,244,247,282,0,2,249,287


upgradeHSTS_subdomain


,brave Android 11 1.62.165 intent real,chrome iPadOS 17.3.1 122.0.6261.89 intent real,firefox Ubuntu 22.04 121 selenium headless,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,54,68,1,57
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,54,68,1,57
chrome Android 11 121.0.6167.180 intent real,0,54,68,1,57
chrome Ubuntu 22.04 120 selenium headless-new,0,54,68,1,57
chrome Ubuntu 22.04 121 selenium headless-new,0,54,68,1,57
chrome Ubuntu 22.04 122 selenium headless-new,0,54,68,1,57
chrome iPadOS 17.3.1 122.0.6261.89 intent real,54,0,80,55,3
edge Ubuntu 22.04 121 selenium headless-new,0,54,68,1,57
firefox Ubuntu 22.04 121 selenium headless,68,80,0,69,83
firefox Ubuntu 22.04 122 selenium headless,68,80,0,69,83


In [79]:
# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_id"], observed=True)["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers. Remaining rows:", len(tree_df))

Remove tests that are the same in all browsers. Remaining rows: 116620


In [78]:
# How many rows exist for tree creation
with pd.option_context("display.max_rows", 100):
    display(tree_df[["test_name", "label", "relation_info"]].value_counts())

test_name                      label       relation_info 
oac_window.open                OAC         window.open       49868
fullscreen_iframe              PP          child_allow        6552
                                           direct             5796
upgradeHSTS_direct             HSTS        direct             4452
imgloading_iframe              CSP-IMG     direct             3402
subresourceloadingCOEP_img     COEP        direct             3346
framing_iframe                 CSP-FA      direct             3066
script_execution_iframe        CSP-SCRIPT  direct             3024
subresourceloadingCORP_object  CORP        direct             2464
oac_iframe                     OAC         sandbox            2016
                                           direct             2016
referrer_iframe                RP          window.open        1680
                                           iframe             1666
framing_iframe                 CSP-FA      nested             1498
upgr

In [88]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()

base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,25 days 21 hours 4 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 11 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,25 days 21 hours 4 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 11 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 1008
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 2464
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,25 days 21 hours 5 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 11 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 4452
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,25 days 21 hours 5 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 11 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 1470


# Manual analysis

## Helpers

In [113]:
# Open Tree SVG
from IPython.display import display, HTML
import os

def open_tree(tree_name):
    directory = 'trees/'
    latest_tree_directory = max([os.path.join(directory, d) for d in os.listdir(directory)], key=os.path.getmtime)
    file_path = f"http://localhost:8888/files/analysis/{latest_tree_directory}/svg/{tree_name}.svg"
    html_link = f'<a href="{file_path}" target="_blank">Open {tree_name}.svg</a>'
    display(HTML(html_link))

open_tree("('accesswindow_direct', 'direct')")

In [119]:
def show_overlap(test_name, groupby=True, show_all=True):
    d = df.loc[df["test_name"] == test_name]
    if groupby:
        for name, group in d.groupby("resp_type"):
            display_overlap(group, show_all=show_all, name=name)
    else:
        display_overlap(d, show_all=show_all, name="all")


In [244]:
from io import StringIO

def show_response_groups(test_name):
    splits = {}
    for test_id, group in tree_df.loc[tree_df["test_name"] == test_name].groupby("test_id", observed=True):
        key = group[["browser", "outcome_str"]].drop_duplicates().sort_values("browser")
        key = key.to_csv(index=False)
        try:
            splits[key].append(test_id)
        except (AttributeError, KeyError):
            splits[key] = [test_id]

    for i, (key, values) in enumerate(splits.items()):
        print(f"Group {i}")
        display(pd.read_csv(StringIO(key)).groupby("outcome_str")["browser"].unique().apply(sorted).to_dict())
        with pd.option_context("display.max_colwidth", 500):
            cur_data = tree_df.loc[tree_df["test_id"].isin(values)]
            display(cur_data[["raw_header", "status_code", "org_origin", "resp_origin", "response_id"]].drop_duplicates().sort_values(by=["raw_header", "status_code", "org_origin", "resp_origin"]))
            example = cur_data.iloc[0]
            example_url = example["clean_url"]
            example_element_relation = example["test_name"].split("_")[1] + "_" + example["relation_info"]
            example_resp_origin = example["resp_origin"]
            example_resp_id = example["response_id"]
            example_url += f"&t_resp_id={example_resp_id}&t_element_relation={example_element_relation}&t_resp_origin={example_resp_origin}"
            display(example_url)
            
            

## accesswindow_direct/COOP window-references-coop.sub.html
- Testfile: `window-references-coop.sub.html`
- Testnames: accesswindow_direct
- Header: COOP
- Basic:
    - Android chrome (121), Ubuntu chrome (120-122), edge (121), brave (121) are identical
    - Android: Opera + brave sligthly different?
    - Safari, Firefox (all platforms) are identical
    - The two groups have 28 differences
- Parsing:
    - Ubuntu chrome (120-122), edge (121), brave (121), opera (Android 121) are identical
    - Android: Brave + chrome identical
    - Firefox desktop identical
    - Safari almost identical to Chrome/Brave Android?
    - ...
- [x] Investigate complex differences:
    - Maybe: same-origin differences due to HTTPS upgrades vs no-upgrade? (basic; chrome auto updates?)
    - Maybe: \n and \r and similar interpreted differently
    - See below!
- [x] Confirm some of the strange cases on Android + iPad (are they noise?) 

In [263]:
test_name = 'accesswindow_direct'

In [114]:
# Tree
open_tree(f"('{test_name}', 'direct')")

In [146]:
df.loc[df["test_name"] == test_name].groupby("test_id", observed=True)["outcome_str"].nunique().value_counts()

outcome_str
1    5416
2      72
Name: count, dtype: int64

In [123]:
show_overlap(test_name, groupby=True, show_all=True)

basic


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,4,4,4,4,4,24,4,24,24,24,24,9,24
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,4,0,0,0,0,0,28,0,28,28,28,28,7,28
chrome Android 11 121.0.6167.180 intent real,4,0,0,0,0,0,28,0,28,28,28,28,7,28
chrome Ubuntu 22.04 120 selenium headless-new,4,0,0,0,0,0,28,0,28,28,28,28,7,28
chrome Ubuntu 22.04 121 selenium headless-new,4,0,0,0,0,0,28,0,28,28,28,28,7,28
chrome Ubuntu 22.04 122 selenium headless-new,4,0,0,0,0,0,28,0,28,28,28,28,7,28
chrome iPadOS 17.3.1 122.0.6261.89 intent real,24,28,28,28,28,28,0,28,0,0,0,0,21,0
edge Ubuntu 22.04 121 selenium headless-new,4,0,0,0,0,0,28,0,28,28,28,28,7,28
firefox Ubuntu 22.04 121 selenium headless,24,28,28,28,28,28,0,28,0,0,0,0,21,0
firefox Ubuntu 22.04 122 selenium headless,24,28,28,28,28,28,0,28,0,0,0,0,21,0


parsing


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real
brave Android 11 1.62.165 intent real,0,11,0,11,11,11,9,11,39,39,39,29,11,1
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,11,0,11,0,0,0,12,0,28,28,28,40,0,12
chrome Android 11 121.0.6167.180 intent real,0,11,0,11,11,11,9,11,39,39,39,29,11,1
chrome Ubuntu 22.04 120 selenium headless-new,11,0,11,0,0,0,12,0,28,28,28,40,0,12
chrome Ubuntu 22.04 121 selenium headless-new,11,0,11,0,0,0,12,0,28,28,28,40,0,12
chrome Ubuntu 22.04 122 selenium headless-new,11,0,11,0,0,0,12,0,28,28,28,40,0,12
chrome iPadOS 17.3.1 122.0.6261.89 intent real,9,12,9,12,12,12,0,12,40,40,40,36,12,8
edge Ubuntu 22.04 121 selenium headless-new,11,0,11,0,0,0,12,0,28,28,28,40,0,12
firefox Ubuntu 22.04 121 selenium headless,39,28,39,28,28,28,40,28,0,0,0,12,28,40
firefox Ubuntu 22.04 122 selenium headless,39,28,39,28,28,28,40,28,0,0,0,12,28,40


In [141]:
df.loc[df["test_name"] == test_name].pivot_table("test_id", "outcome_str", aggfunc="nunique")

,test_id
outcome_str,
{'window.open.opener': 'null'},196
"{'window.open.opener': 'object ""[object Window]""'}",5364


In [249]:
tree_df.loc[tree_df["test_name"] == test_name].pivot_table("test_id", "outcome_str", aggfunc="nunique")

,test_id
outcome_str,
{'window.open.opener': 'null'},72
"{'window.open.opener': 'object ""[object Window]""'}",72


In [136]:
with pd.option_context("display.max_colwidth", None):
    display(tree_df.loc[tree_df["test_name"] == test_name].groupby(["outcome_str", "browser"], observed=True)["id"].count().unstack(0))

outcome_str,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
browser,,
brave Android 11 1.62.165 intent real,41,31
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,56,16
chrome Android 11 121.0.6167.180 intent real,45,27
chrome Ubuntu 22.04 120 selenium headless-new,56,16
chrome Ubuntu 22.04 121 selenium headless-new,56,16
chrome Ubuntu 22.04 122 selenium headless-new,56,16
chrome iPadOS 17.3.1 122.0.6261.89 intent real,24,48
edge Ubuntu 22.04 121 selenium headless-new,56,16
firefox Ubuntu 22.04 121 selenium headless,24,48


In [148]:
# All response headers that (sometimes) result in different outcomes
tree_df.loc[tree_df["test_name"] == test_name].groupby("resp_type")["raw_header"].unique().to_list()

[array(["[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",
        "[['Cross-Origin-Opener-Policy', 'same-origin']]",
        "[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",
        "[['Cross-Origin-Opener-Policy', 'same-origin'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]"],
       dtype=object),
 array(["[['\\ncross-origin-opener-policy', 'unsafe-none']]",
        "[['cross-origin-opener-policy ', 'same-origin']]",
        "[['\\ncross-origin-opener-policy', 'same-origin']]",
        "[['cross-origin-opener-policy  ', 'same-origin']]",
        "[['cross-origin-opener-policy\\t', 'same-origin']]",
        "[['cross-origin-opener-policy\\r', 'same-origin']]",
        "[['\\rcross-origin-opener-policy', 'same-origin']]",
        "[['cross-origin-opener-policy ', 'same-origin-allow-popups']]",
        "[['\\ncross-origin-opener-policy', 'same-origin-allo

In [266]:
# Group 0: COOP only active on HTTPS, even though resp_origin is HTTP COOP is active in chromium-based browsers due to HTTPS auto-upgrade; Opera wrong group due to noise?
# Group 1: same; Opera correct
# Group 2: download triggered due to \n in response. Safari (+ Android without Opera?) download pages are treated similarily to about:blank?
# Group 3: \r, \t, or space at the beginning or end of header name allowed in all non-firefox browsers; real finding? which behavior is correct?
# Group 4: Two different findings; 1. Two headers, one empty: COOP active in Firefox, not-active in other browsers (which behavior is correct?), 2. \r at the end of the header name: works in Firefox, does not work in the other browsers!
# Group 5: Status code 204 + COOP works on iPad; does not work in other browsers (bug?)
# Group 6: Same/similar to group 2? (only iPad is on the other group?)
# Group 7: Similar to 0 and 1? Brave Android maybe in the incorrect group for some reason?
# Group 8: Similar to 0, 1, 7? Brave Android + opera maybe in the wrong group?
# Group 9: Similar to group 2 and 6? 2 Android browsers are in the other group however (might be noise?)

# Noise check/Manual confirmation:
# Android: http://134.96.225.54:8888/#!/control/emulator-5554

# Difficult?!
# Group 0 opera appears to be in the other group, recorded result was noise?
# Group 1 confirmed
# Group 7 brave is in the firefox group?, recorded result was no noise? (what is the difference to 0-1?); however opera is also in the firefox group (recorded result was noise?)
# Group 8 opera + brave stay the same/no noise; actual difference might be: only chrome applies redirect + COOP!
# Difference between (0,1) and (7,8) is that the latter uses http://sub.headers.websec.saarland; seems like opera + brave do not auto-update this origin (exception?)

# Group 2: confirmed
# Group 6: confirmed iPad is in the other group; not sure why? probably COOP applies before the download? (the difference between 2 is that \n is at the end, and the difference to 9 is that the COOP value is blocking)
# Group 9: all Android (except Opera) in the other group (noise in the results?, brave + chrome in wrong group!)

# Final Groups?:
# 0,1 (opera had noise)
# 2,9 (brave + chrome had noise)
# 3
# 4 (actual two different things in this group)
# 5
# 6
# 7,8 (opera had noise)

show_response_groups(test_name)

Group 0


{"{'window.open.opener': 'null'}": ['brave Android 11 1.62.165 intent real',
  'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new',
  'chrome Android 11 121.0.6167.180 intent real',
  'chrome Ubuntu 22.04 120 selenium headless-new',
  'chrome Ubuntu 22.04 121 selenium headless-new',
  'chrome Ubuntu 22.04 122 selenium headless-new',
  'edge Ubuntu 22.04 121 selenium headless-new'],
 '{\'window.open.opener\': \'object "[object Window]"\'}': ['chrome iPadOS 17.3.1 122.0.6261.89 intent real',
  'firefox Ubuntu 22.04 121 selenium headless',
  'firefox Ubuntu 22.04 122 selenium headless',
  'firefox Ubuntu 22.04 123 selenium headless',
  'firefox_beta Android 11 123.0b9 intent real',
  'opera Android 11 80.5.4244.78163 intent real',
  'safari macOS 14.3.1 17.3.1 selenium real']}

,raw_header,status_code,org_origin,resp_origin,response_id
140,"[['Cross-Origin-Opener-Policy', 'same-origin'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",302,http://sub.headers.websec.saarland,http://headers.webappsec.eu,81
602,"[['Cross-Origin-Opener-Policy', 'same-origin'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",302,http://sub.headers.websec.saarland,http://sub.sub.headers.websec.saarland,81
28,"[['Cross-Origin-Opener-Policy', 'same-origin']]",200,http://sub.headers.websec.saarland,http://headers.webappsec.eu,73
126,"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",302,http://sub.headers.websec.saarland,http://headers.webappsec.eu,80
588,"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",302,http://sub.headers.websec.saarland,http://sub.sub.headers.websec.saarland,80
14,"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",200,http://sub.headers.websec.saarland,http://headers.webappsec.eu,72


'http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http&t_resp_id=72&t_element_relation=direct_direct&t_resp_origin=http://headers.webappsec.eu'

Group 1


...

'https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=80&last_id=80&scheme=https&t_resp_id=80&t_element_relation=direct_direct&t_resp_origin=http://sub.headers.websec.saarland'

Group 9


{"{'window.open.opener': 'null'}": ['brave Android 11 1.62.165 intent real',
  'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new',
  'chrome Android 11 121.0.6167.180 intent real',
  'chrome Ubuntu 22.04 120 selenium headless-new',
  'chrome Ubuntu 22.04 121 selenium headless-new',
  'chrome Ubuntu 22.04 122 selenium headless-new',
  'edge Ubuntu 22.04 121 selenium headless-new',
  'firefox Ubuntu 22.04 121 selenium headless',
  'firefox Ubuntu 22.04 122 selenium headless',
  'firefox Ubuntu 22.04 123 selenium headless',
  'opera Android 11 80.5.4244.78163 intent real'],
 '{\'window.open.opener\': \'object "[object Window]"\'}': ['chrome iPadOS 17.3.1 122.0.6261.89 intent real',
  'firefox_beta Android 11 123.0b9 intent real',
  'safari macOS 14.3.1 17.3.1 selenium real']}

,raw_header,status_code,org_origin,resp_origin,response_id
61852,"[['cross-origin-opener-policy', 'unsafe-none\n']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,23242


'https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=parsing&browser_id=1&label=COOP&first_id=23242&last_id=23242&scheme=https&t_resp_id=23242&t_element_relation=direct_direct&t_resp_origin=https://headers.webappsec.eu'

In [248]:
# How often do various status codes exist?
df.loc[df["test_name"] == test_name].groupby("browser")["status_code"].value_counts().to_frame()

count
browser                                  status_code       
brave Android 11 1.62.165 intent real    200           5396
                                         302             38
                                         203              6
                                         204              6
                                         300              6
...                                                     ...
safari macOS 14.3.1 17.3.1 selenium real 400              6
                                         403              6
                                         404              6
                                         500              6
                                         418              6

[154 rows x 1 columns]

## Fetch/CORS 
- Testfile: `fetch-cors.sub.html`
- Testnames: fetch_GET_credentials|custom_headers_simple, fetch_TEST_custom_method
- Headers: ACAO, ACAM, ACAH, ACAC, ACEH
- Basic + Parsing, ...

In [252]:
test_name = 'fetch_GET' # fetch_TEST

In [115]:
# Trees
open_tree("('fetch_GET', 'credentials')")
open_tree("('fetch_GET', 'custom_headers')")
open_tree("('fetch_GET', 'simple')")
open_tree("('fetch_TEST', 'custom_method')")

In [259]:
df.loc[df["test_name"] == test_name].pivot_table("test_id", "outcome_str", aggfunc="nunique")

,test_id
outcome_str,
"{'error': 'null', 'headers': ''}",3
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",1
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",91
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-origin,content-length,date,server,test,'}",6
"{'error': 'null', 'headers': 'access-control-allow-credentials,content-length,date,server,test,'}",6
"{'error': 'null', 'headers': 'access-control-allow-origin,cache-control,content-type,date,server,'}",6
"{'error': 'null', 'headers': 'access-control-allow-origin,content-length,date,server,test,'}",6
"{'error': 'null', 'headers': 'cache-control,content-type,'}",35
"{'error': 'null', 'headers': 'content-length,'}",4516


In [254]:
tree_df.loc[tree_df["test_name"] == test_name].pivot_table("test_id", "outcome_str", aggfunc="nunique")

,test_id
outcome_str,
"{'error': 'null', 'headers': ''}",3
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",1
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",12
"{'error': 'null', 'headers': 'cache-control,content-type,'}",10
"{'error': 'null', 'headers': 'content-length,'}",49
"{'error': 'null', 'headers': 'content-length,test,'}",139
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",138
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",266


In [261]:
# All response headers that (sometimes) result in different outcomes
tree_df.loc[tree_df["test_name"] == test_name].groupby("resp_type")["raw_header"].unique().to_list()

[array(["[['\\naccess-control-allow-credentials', 'true'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
        "[['access-control-allow-credentials', 'true\\n'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
        "[['\\naccess-control-allow-methods', '*'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
        "[['\\naccess-control-allow-methods', 'TEST'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-c

In [260]:
# TODO: maybe differentiate between the various fetch_GET subtests?
# Group 0: ...
# ...
show_response_groups(test_name)

Group 0


{'{\'error\': \'object "AbortError: Fetch is aborted"\', \'headers\': \'\'}': ['firefox Ubuntu 22.04 121 selenium headless',
  'firefox Ubuntu 22.04 122 selenium headless',
  'firefox Ubuntu 22.04 123 selenium headless',
  'firefox_beta Android 11 123.0b9 intent real'],
 '{\'error\': \'object "TypeError: Failed to fetch"\', \'headers\': \'\'}': ['brave Android 11 1.62.165 intent real',
  'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new',
  'chrome Android 11 121.0.6167.180 intent real',
  'chrome Ubuntu 22.04 120 selenium headless-new',
  'chrome Ubuntu 22.04 121 selenium headless-new',
  'chrome Ubuntu 22.04 122 selenium headless-new',
  'chrome iPadOS 17.3.1 122.0.6261.89 intent real',
  'edge Ubuntu 22.04 121 selenium headless-new',
  'opera Android 11 80.5.4244.78163 intent real',
  'safari macOS 14.3.1 17.3.1 selenium real']}

,raw_header,status_code,org_origin,resp_origin,response_id
78008,"[['\naccess-control-allow-credentials', 'true'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,http://sub.headers.websec.saarland,https://headers.webappsec.eu,10060
191506,"[['\naccess-control-allow-credentials', 'true'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,10060
113806,"[['\naccess-control-allow-headers', '*'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-expose-headers', 'Test']]",200,http://sub.headers.websec.saarland,https://headers.webappsec.eu,12617
227304,"[['\naccess-control-allow-headers', '*'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-expose-headers', 'Test']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,12617
119658,"[['\naccess-control-allow-headers', 'Test'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-expose-headers', 'Test']]",200,http://sub.headers.websec.saarland,https://headers.webappsec.eu,13035
233156,"[['\naccess-control-allow-headers', 'Test'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-expose-headers', 'Test']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,13035
92344,"[['\naccess-control-allow-methods', '*'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,http://sub.headers.websec.saarland,https://headers.webappsec.eu,11084
205842,"[['\naccess-control-allow-methods', '*'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,11084
98196,"[['\naccess-control-allow-methods', 'TEST'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,http://sub.headers.websec.saarland,https://headers.webappsec.eu,11502
211694,"[['\naccess-control-allow-methods', 'TEST'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",200,https://sub.headers.websec.saarland,https://headers.webappsec.eu,11502


'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAC&first_id=10059&last_id=10063&scheme=http&t_resp_id=10060&t_element_relation=GET_credentials&t_resp_origin=https://headers.webappsec.eu'

Group 1


...

'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAO&first_id=9422&last_id=9426&scheme=http&t_resp_id=9424&t_element_relation=GET_simple&t_resp_origin=https://headers.webappsec.eu'

Group 14


{"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}": ['brave Android 11 1.62.165 intent real',
  'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new',
  'chrome Android 11 121.0.6167.180 intent real',
  'chrome Ubuntu 22.04 120 selenium headless-new',
  'chrome Ubuntu 22.04 121 selenium headless-new',
  'chrome Ubuntu 22.04 122 selenium headless-new',
  'chrome iPadOS 17.3.1 122.0.6261.89 intent real',
  'edge Ubuntu 22.04 121 selenium headless-new',
  'opera Android 11 80.5.4244.78163 intent real',
  'safari macOS 14.3.1 17.3.1 selenium real'],
 "{'error': 'null', 'headers': 'cache-control,content-type,'}": ['firefox Ubuntu 22.04 121 selenium headless',
  'firefox Ubuntu 22.04 122 selenium headless',
  'firefox Ubuntu 22.04 123 selenium headless',
  'firefox_beta Android 11 123.0b9 intent real']}

,raw_header,status_code,org_origin,resp_origin,response_id
713342,"[['access-control-expose-headers', '*'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test']]",300,https://sub.headers.websec.saarland,https://headers.webappsec.eu,14906


'https://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACEH&first_id=14906&last_id=14910&scheme=https&t_resp_id=14906&t_element_relation=GET_simple&t_resp_origin=https://headers.webappsec.eu'

## Framing
- ...

## Fullscreen/PP
- ...

## Imgloading/CSP-subresources
- ...

## OAC
- ...

## PerformanceAPI
- ...

## Referer/Referrer Policy
- ...

## Script execution/CSP
- ...

## COEP
- ...

## CORP
- ...

## HSTS
- ...

## Next category!

In [89]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

count
clean_url                                          raw_header                                         outcome_str browser                                                  
http://sub.headers.websec.saarland/_hp/tests/su... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.62.165 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 121.0.6167.180 intent real            1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  firefox Ubuntu 22.04 121 selenium headless              1
                                                                                                                  firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  firefox Ubuntu 22.04 123 selenium headless              1
                                                                                                                  firefox_beta Android 11 123.0b9 intent real             1
                                                                                                                  opera Android 11 80.5.4244.78163 intent real            1
                                                                                                                  safari macOS 14.3.1 17.3.1 selenium real                1
https://sub.headers.websec.saarland/_hp/tests/s... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.62.165 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 121.0.6167.180 intent real            1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  opera Android 11 80.5.4244.78163 intent real            1
                                      

## Browser comparisons
- One to one browser comparisons
- E.g., what difference exist between Chrome 121 and Chrome 122

In [90]:
bf = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")

def com_browsers(df, b1, b2):
    bf["diff"] = bf[b1] != bf[b2]
    with pd.option_context("display.max_colwidth", None):
        display(bf.loc[bf["diff"]][[b1, b2]])

### Chromium 122 vs 121 vs 120 (desktop linux)
- Basic RP test: referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic
- 'Referrer-Policy', 'origin-when-cross-origin'
- 120 has full URL, 121 and 122 report `document.referrer: http://sub.headers.websec.saarland/`

In [96]:
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 122 selenium headless-new")
df.loc[df["response_id"] == 196]["raw_header"].iloc[0]

browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new
test_id,,
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


"[['Referrer-Policy', 'origin-when-cross-origin']]"

### Firefox 120 vs 121 (desktop linux)
- subresourceloadingCOEP_img_direct both basic and parsing
- random behavior?

In [97]:
com_browsers(bf, "firefox Ubuntu 22.04 121 selenium headless", "firefox Ubuntu 22.04 122 selenium headless")

browser,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
test_id,,
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_62_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_63_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_64_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_62_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_63_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_64_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_62_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_63_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_64_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",

"[['Cross-Origin-Embedder-Policy', '']]"

### Brave 121 vs Chrome 121
- All about referrer/RP
- Most of the time brave has less info in the referrer (e.g., does not support unsafe-url)
- `referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic` here Brave leaks the full_url whereas chrome only has the origin?!
- 196: `'Referrer-Policy', 'origin-when-cross-origin'`
- Chrome 120 also leaks the full URL!

In [100]:
display(df.loc[df["response_id"] == 196]["raw_header"].iloc[0])
with pd.option_context("display.max_rows", 142):
    com_browsers(bf, "brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")

"[['Referrer-Policy', 'origin-when-cross-origin']]"

browser,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_192_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_198_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_200_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_192_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_198_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_200_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_192_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_198_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_200_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url


### Safari (MacOS 17.3.1) vs Chrome/WebKit (iPadOS 17.3.1)
- Wait until noise in Safari is filtered out?

In [102]:
with pd.option_context("display.max_rows", 237):
    com_browsers(bf, "chrome iPadOS 17.3.1 122.0.6261.89 intent real", "safari macOS 14.3.1 17.3.1 selenium real")

browser,chrome iPadOS 17.3.1 122.0.6261.89 intent real,safari macOS 14.3.1 17.3.1 selenium real
test_id,,
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_23342_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_23606_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_23762_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_23990_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_23342_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_23606_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_23762_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_23990_parsing,{'window.open.opener': 'null'},"{'window.open.opener': 'object ""[object Window]""'}"
fetch_GET_credentials_http_sub.headers.websec.saarland_https_headers.webappsec.eu_14967_parsing,"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}","['{\'error\': \'object ""AbortError: Fetch is aborted""\', \'headers\': \'\'}'\n '{\'error\': \'object ""TypeError: Failed to fetch""\', \'headers\': \'\'}']"


## Missing tests 
- None missing for main browsers
- Other Android browsers?!

In [95]:
# Missing tests (other Android browsers?)

pivot_df = df.pivot_table(index='test_name', columns='browser', values='test_id', aggfunc='count', fill_value=0)
# Calculate the difference between the counts of the first and second columns
pivot_df['Diff'] = pivot_df.iloc[:, 0] - pivot_df.iloc[:, 1]
display(pivot_df)

browser,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real,safari macOS 14.3.1 17.3.1 selenium real,Diff
test_name,,,,,,,,,,,,,,,
accesswindow_direct,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,0
fetch_GET,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,0
fetch_TEST,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,0
framing_embed,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
framing_iframe,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,0
framing_object,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
fullscreen_iframe,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,0
imgloading_iframe,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,0
oac_iframe,448,448,448,448,448,448,448,448,448,448,448,448,448,448,0
